In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('data/df_ml.csv', sep=",")
#df['Date'] = pd.to_datetime(df['Date'])

#df.drop(index=1, inplace=True)
df.head(3)

,Open,High,Low,Close,Adj Close,Volume($),Symbol,Daily Return,SMA_30,SMA_60,...,Close_rolling_std_7,Volume_rolling_mean_7,Volume_rolling_std_7,Close_rolling_mean_30,Close_rolling_std_30,Volume_rolling_mean_30,Volume_rolling_std_30,Year,Month,Day
0,28994.009766,29600.626953,28803.585938,29374.152344,29374.152344,40730301359,BTC,0.012842,22428.243750,19715.608773,...,1284.059118,4.970113e+10,8.115395e+09,22428.243750,3661.566866,3.886562e+10,1.223869e+10,2021,1,1
1,29376.455078,33155.117188,29091.181641,32127.267578,32127.267578,67865420765,BTC,0.093726,22850.972721,20018.558219,...,1936.633106,5.249153e+10,1.055716e+10,22850.972721,4019.854985,4.006346e+10,1.325301e+10,2021,1,2
2,32129.408203,34608.558594,32052.316406,32782.023438,32782.023438,78665235202,BTC,0.020380,23320.381315,20329.363493,...,2189.909798,5.423229e+10,1.376529e+10,23320.381315,4328.735995,4.155655e+10,1.494648e+10,2021,1,3


In [3]:
df = df[['Close','Symbol', 'Close_lag_1', 'Low','Midpoint','High','Open','Close_lag_3']] #
#df = df[['Close','Symbol','Close_lag_7','EMA_90', '90_day_MA', 'Close_rolling_mean_30','EMA_60','EMA_26','SMA_30']]
#df.head(1)



28             Close_lag_7        1
12                  EMA_90        1
23               90_day_MA        1
34   Close_rolling_mean_30        1
11                  EMA_60        2
15                  EMA_26        3
7                   SMA_30        4

#### Onehot encoding 

In [4]:
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)


artists_column = df['Symbol'].values.reshape(-1, 1)

one_hot_encoded_artists = one_hot_encoder.fit_transform(artists_column)

df['Symbol'] = np.argmax(one_hot_encoded_artists, axis=1)

#### Scaling and Spliting train,test,validation

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data into features (X) and target variable (y)
X = df.drop('Close', axis=1)  
y = df['Close']

# Split the dataset into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.fit_transform(X)


## Machine Learning Models

### Linear Regression

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Initialize the model
linear_reg = LinearRegression()


linear_reg.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = linear_reg.predict(X_test_scaled)

mse_lr = mean_squared_error(y_test, y_pred)
print(f"Linear Regression MSE: {mse_lr}")


Linear Regression MSE: 19942.26454936525


In [7]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Calculate R-squared
r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

R-squared: 0.9998187610689385
Mean Absolute Error (MAE): 30.31806788784291
Root Mean Squared Error (RMSE): 141.2170830649226


##### Cross-Validation for Linear Regression

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

# Initialize the model
linear_reg = LinearRegression()

# Perform cross-validation
cv_scores_lr = cross_val_score(linear_reg, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Calculate the mean MSE
mean_mse_lr = -cv_scores_lr.mean()
print(f"Linear Regression Cross-Validated MSE: {mean_mse_lr}")


Linear Regression Cross-Validated MSE: 21423.14803174241


#### Grid Search
Grid Search evaluates all the provided combinations of hyperparameters, which can be computationally expensive but thorough.

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the model
rf = RandomForestRegressor()

# Set up the hyperparameters grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Set up the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", -grid_search.best_score_)


KeyboardInterrupt: 

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [10, 20, 30, None],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at a leaf node
    'bootstrap': [True, False]  # Method of selecting samples for training each tree
}

# Initialize the Grid Search model
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train_scaled, y_train)

# Best parameters found by Grid Search
print("Best parameters found: ", grid_search.best_params_)

# Best score found by Grid Search
print("Best score found: ", -grid_search.best_score_)





Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters found:  {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best score found:  38483.10625303709


In [ ]:
# Initialize the model with the best parameters
best_params = grid_search.best_params_
optimized_random_forest = RandomForestRegressor(**best_params, random_state=42)

# Fit the model with the training data
optimized_random_forest.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_optimized_rf = optimized_random_forest.predict(X_test_scaled)

# Evaluate the model with the test data
optimized_mse_rf = mean_squared_error(y_test, y_pred_optimized_rf)
print(f"Optimized Random Forest MSE: {optimized_mse_rf}")


### Decision Trees and Random Forest

In [190]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model with the best parameters from Grid Search
random_forest = RandomForestRegressor(
    n_estimators=100,  # Best number of trees
    max_depth=30,  # Best maximum depth of trees
    min_samples_leaf=1,  # Best minimum number of samples required at a leaf node
    min_samples_split=2,  # Best minimum number of samples required to split an internal node
    random_state=42  # For reproducibility
)

# Fit the model with the training data
random_forest.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_rf = random_forest.predict(X_test_scaled)

# Evaluate the model with the test data
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest MSE with Best Parameters: {mse_rf}")


Random Forest MSE with Best Parameters: 33965.6070107671


In [230]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Calculate R-squared
r_squared = r2_score(y_test, y_pred_rf)
print(f"R-squared: {r_squared}")

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred_rf)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f"Root Mean Squared Error (RMSE): {rmse}")

R-squared: 0.9996913143794554
Mean Absolute Error (MAE): 40.66735938525787
Root Mean Squared Error (RMSE): 184.29760446290965


##### Cross-Validation for Random Forest

In [231]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform cross-validation
cv_scores_rf = cross_val_score(random_forest, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Calculate the mean MSE
mean_mse_rf = -cv_scores_rf.mean()
print(f"Random Forest Cross-Validated MSE: {mean_mse_rf}")


Random Forest Cross-Validated MSE: 96922514.69028388


### Gradient Boosting Machines (GBM)

#### Random Search
Random Search samples a given number of candidates from a parameter space with a specified distribution. It's less comprehensive but much faster than Grid Search.

In [223]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Define the parameter distribution to sample from
param_dist = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'colsample_bytree': [0.3, 0.7],
    'subsample': [0.6, 0.9]
}

# Initialize the XGBRegressor
xg_reg = xgb.XGBRegressor()

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xg_reg, param_distributions=param_dist, n_iter=25, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, random_state=42)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters found: ", random_search.best_params_)
print("Best score found: ", -random_search.best_score_)


Best parameters found:  {'subsample': 0.6, 'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 0.7}
Best score found:  39292.73854500158


In [234]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Adjusting the model with the best parameters found
xg_reg_optimized = xgb.XGBRegressor(
    objective='reg:squarederror',
    colsample_bytree=0.7,  # Updated from 0.3 to 0.7 based on the best parameters
    learning_rate=0.1,     # Remains the same as before
    max_depth=5,           # Remains the same as before
    alpha=10,              # Remains the same as before
    n_estimators=500,      # Updated from 10 to 500 based on the best parameters
    subsample=0.8          # Added based on the best parameters
)

# Fit the model with the adjusted parameters
xg_reg_optimized.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_optimized = xg_reg_optimized.predict(X_test_scaled)

# Evaluate the model with the adjusted parameters
mse_xgb_optimized = mean_squared_error(y_test, y_pred_optimized)
print(f"Optimized XGBoost MSE: {mse_xgb_optimized}")


Optimized XGBoost MSE: 32797.58750307243


In [235]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Calculate R-squared
r_squared = r2_score(y_test, y_pred_optimized)
print(f"R-squared: {r_squared}")

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred_optimized)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_optimized))
print(f"Root Mean Squared Error (RMSE): {rmse}")

R-squared: 0.9997019295533995
Mean Absolute Error (MAE): 39.37663597807381
Root Mean Squared Error (RMSE): 181.10104224733888


##### Cross-Validation for XGBoost

In [236]:
import xgboost as xgb

# Initialize the model
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                max_depth=5, alpha=10, n_estimators=10)

# Perform cross-validation
cv_scores_xgb = cross_val_score(xg_reg, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Calculate the mean MSE
mean_mse_xgb = -cv_scores_xgb.mean()
print(f"XGBoost Cross-Validated MSE: {mean_mse_xgb}")


XGBoost Cross-Validated MSE: 98328661.26877618


### LazyPredict

In [13]:
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import shuffle

# Shuffle your dataset (optional but recommended for cross-sectional data)
#X, y = shuffle(X, y, random_state=42)

# Split the dataset into training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize LazyRegressor
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit and compare models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

# Print the performance of each model
print(models)


 74%|███████▍  | 31/42 [01:05<00:13,  1.18s/it]

In [ ]:
df.to_csv('data/df_fs.csv', index=False)